In [1]:
from sklearn.datasets import load_iris
import numpy as np
import math
from tqdm import tqdm
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import pandas as pd
from sklearn.preprocessing import (
    LabelEncoder,
    OneHotEncoder,
)
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix

In [2]:
model = LogisticRegression(C=0.5)

In [3]:
iden = pd.read_csv('train_identity.csv')

In [4]:
trans = pd.read_csv('train_transaction.csv')

In [5]:
joined = trans.merge(iden,how='left')

In [6]:
del iden

In [7]:
del trans

In [8]:
joined = joined[joined['TransactionDT'].notnull()]

In [9]:
len(joined)

590540

In [10]:
limite = np.percentile(joined['TransactionDT'], 80)

In [11]:
limite

12192853.600000001

In [12]:
joined['TransactionDT'].value_counts()

9474817     8
4397066     5
11576951    5
7236588     5
8218707     4
           ..
11481962    1
14956586    1
12853289    1
7622695     1
9943554     1
Name: TransactionDT, Length: 573349, dtype: int64

In [13]:
filtro1 = joined['TransactionDT'] > limite

In [14]:
validation = joined[filtro1]

In [15]:
validation

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
472432,3459432,1,12192900,33.261,C,9300,103.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,NaN
472433,3459433,0,12192911,52.811,C,8809,179.0,106.0,visa,137.0,...,chrome generic for android,NaN,NaN,NaN,F,F,T,F,mobile,F3311
472434,3459434,0,12192913,136.956,C,10819,555.0,185.0,visa,226.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472435,3459435,0,12193040,136.956,C,9633,130.0,185.0,visa,138.0,...,chrome 65.0,NaN,NaN,NaN,F,F,T,F,desktop,Windows
472436,3459436,0,12193199,25.000,H,17188,321.0,150.0,visa,226.0,...,firefox 59.0,24.0,3200x1800,match_status:2,T,F,T,T,desktop,Windows
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
590535,3577535,0,15811047,49.000,W,6550,NaN,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590536,3577536,0,15811049,39.500,W,10444,225.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590537,3577537,0,15811079,30.950,W,12037,595.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
590538,3577538,0,15811088,117.000,W,7826,481.0,150.0,mastercard,224.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
filtro2 = joined['TransactionDT']<= limite

In [17]:
train = joined[filtro2]

In [18]:
train

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,0,86400,68.50,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.00,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,0,12192667,43.95,W,15484,418.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472428,3459428,0,12192736,49.00,W,17188,321.0,150.0,visa,226.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472429,3459429,0,12192742,40.00,H,16659,170.0,150.0,visa,226.0,...,chrome 65.0,24.0,1440x900,match_status:2,T,F,T,T,desktop,MacOS
472430,3459430,0,12192743,15.00,W,7919,194.0,150.0,mastercard,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
label = train[['isFraud']]

In [20]:
del(train['isFraud'])

In [21]:
train

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987000,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,12192667,43.95,W,15484,418.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472428,3459428,12192736,49.00,W,17188,321.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
472429,3459429,12192742,40.00,H,16659,170.0,150.0,visa,226.0,credit,...,chrome 65.0,24.0,1440x900,match_status:2,T,F,T,T,desktop,MacOS
472430,3459430,12192743,15.00,W,7919,194.0,150.0,mastercard,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
del joined

In [23]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 472432 entries, 0 to 472431
Columns: 433 entries, TransactionID to DeviceInfo
dtypes: float64(399), int64(3), object(31)
memory usage: 1.5+ GB


In [24]:
ohe = OneHotEncoder()

In [25]:
id_34_encoded = (ohe.fit_transform(train[['id_34']].astype(str)).todense().astype(int))
id_34_encoded = pd.DataFrame(id_34_encoded).add_prefix('id_34_')
train = pd.concat([train, id_34_encoded], axis=1)

In [26]:
train['id_34'].fillna(0,inplace=True)

In [27]:
id_35_encoded = (ohe.fit_transform(train[['id_35']].astype(str)).todense().astype(int))
id_35_encoded = pd.DataFrame(id_35_encoded).add_prefix('id_35_')
train = pd.concat([train, id_35_encoded], axis=1)

In [28]:
train['id_35'].fillna(0,inplace=True)

In [29]:
id_38_encoded = (ohe.fit_transform(train[['id_38']].astype(str)).todense().astype(int))
id_38_encoded = pd.DataFrame(id_38_encoded).add_prefix('id_38_')
train = pd.concat([train, id_38_encoded], axis=1)

In [30]:
train['id_38'].fillna(0,inplace=True)

In [31]:
ProductCD_encoded = (ohe.fit_transform(train[['ProductCD']].astype(str)).todense().astype(int))
ProductCD_encoded = pd.DataFrame(ProductCD_encoded).add_prefix('ProductCD_')
train = pd.concat([train, ProductCD_encoded], axis=1)

In [32]:
train['ProductCD'].fillna(0,inplace=True)

In [33]:
P_emaildomain_encoded = (ohe.fit_transform(train[['P_emaildomain']].astype(str)).todense().astype(int))
P_emaildomain_encoded = pd.DataFrame(P_emaildomain_encoded).add_prefix('P_emaildomain_')
train = pd.concat([train, P_emaildomain_encoded], axis=1)

In [34]:
train['P_emaildomain'].fillna(0,inplace=True)

In [35]:
card4_encoded = (ohe.fit_transform(train[['card4']].astype(str)).todense().astype(int))
card4_encoded = pd.DataFrame(card4_encoded).add_prefix('card4')
train = pd.concat([train, card4_encoded], axis=1)

In [36]:
train['card4'].fillna(0,inplace=True)

In [37]:
train

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,P_emaildomain_55,P_emaildomain_56,P_emaildomain_57,P_emaildomain_58,P_emaildomain_59,card40,card41,card42,card43,card44
0,2987000,86400,68.50,W,13926,NaN,150.0,discover,142.0,credit,...,0,0,0,0,0,0,1,0,0,0
1,2987001,86401,29.00,W,2755,404.0,150.0,mastercard,102.0,credit,...,0,0,0,0,0,0,0,1,0,0
2,2987002,86469,59.00,W,4663,490.0,150.0,visa,166.0,debit,...,0,0,0,0,0,0,0,0,0,1
3,2987003,86499,50.00,W,18132,567.0,150.0,mastercard,117.0,debit,...,0,0,0,0,0,0,0,1,0,0
4,2987004,86506,50.00,H,4497,514.0,150.0,mastercard,102.0,credit,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,12192667,43.95,W,15484,418.0,150.0,visa,226.0,debit,...,0,0,0,0,0,0,0,0,0,1
472428,3459428,12192736,49.00,W,17188,321.0,150.0,visa,226.0,debit,...,0,0,0,0,0,0,0,0,0,1
472429,3459429,12192742,40.00,H,16659,170.0,150.0,visa,226.0,credit,...,0,0,0,0,0,0,0,0,0,1
472430,3459430,12192743,15.00,W,7919,194.0,150.0,mastercard,166.0,debit,...,0,0,0,0,0,0,0,1,0,0


In [38]:
le = LabelEncoder()
for column in train.columns:
    if train[column].dtype==object:
        train[column] = le.fit_transform(train[column].astype(str))

In [39]:
train.fillna(-1,inplace=True)

In [40]:
train

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,P_emaildomain_55,P_emaildomain_56,P_emaildomain_57,P_emaildomain_58,P_emaildomain_59,card40,card41,card42,card43,card44
0,2987000,86400,68.50,4,13926,-1.0,150.0,2,142.0,1,...,0,0,0,0,0,0,1,0,0,0
1,2987001,86401,29.00,4,2755,404.0,150.0,3,102.0,1,...,0,0,0,0,0,0,0,1,0,0
2,2987002,86469,59.00,4,4663,490.0,150.0,4,166.0,2,...,0,0,0,0,0,0,0,0,0,1
3,2987003,86499,50.00,4,18132,567.0,150.0,3,117.0,2,...,0,0,0,0,0,0,0,1,0,0
4,2987004,86506,50.00,1,4497,514.0,150.0,3,102.0,1,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
472427,3459427,12192667,43.95,4,15484,418.0,150.0,4,226.0,2,...,0,0,0,0,0,0,0,0,0,1
472428,3459428,12192736,49.00,4,17188,321.0,150.0,4,226.0,2,...,0,0,0,0,0,0,0,0,0,1
472429,3459429,12192742,40.00,1,16659,170.0,150.0,4,226.0,1,...,0,0,0,0,0,0,0,0,0,1
472430,3459430,12192743,15.00,4,7919,194.0,150.0,3,166.0,2,...,0,0,0,0,0,0,0,1,0,0


In [41]:
model.fit(train, label)

C:\Users\julip\anaconda3\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)
C:\Users\julip\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression(C=0.5)

In [42]:
print(classification_report(label, model.predict(train)))

              precision    recall  f1-score   support

           0       0.96      1.00      0.98    455833
           1       0.18      0.00      0.00     16599

    accuracy                           0.96    472432
   macro avg       0.57      0.50      0.49    472432
weighted avg       0.94      0.96      0.95    472432



In [43]:
roc_auc_score(label, model.predict_proba(train)[:,1])

0.7169407566346255

### VALIDATION

In [44]:
id_34_encoded = (ohe.fit_transform(validation[['id_34']].astype(str)).todense().astype(int))
id_34_encoded = pd.DataFrame(id_34_encoded).add_prefix('id_34_')
train = pd.concat([validation, id_34_encoded], axis=1)

In [45]:
validation['id_34'].fillna(0,inplace=True)

In [46]:
id_35_encoded = (ohe.fit_transform(validation[['id_35']].astype(str)).todense().astype(int))
id_35_encoded = pd.DataFrame(id_35_encoded).add_prefix('id_35_')
train = pd.concat([validation, id_35_encoded], axis=1)

In [47]:
validation['id_35'].fillna(0,inplace=True)

In [48]:
id_38_encoded = (ohe.fit_transform(validation[['id_38']].astype(str)).todense().astype(int))
id_38_encoded = pd.DataFrame(id_38_encoded).add_prefix('id_38_')
train = pd.concat([validation, id_38_encoded], axis=1)

In [49]:
validation['id_38'].fillna(0,inplace=True)

In [50]:
ProductCD_encoded = (ohe.fit_transform(validation[['ProductCD']].astype(str)).todense().astype(int))
ProductCD_encoded = pd.DataFrame(ProductCD_encoded).add_prefix('ProductCD_')
train = pd.concat([validation, ProductCD_encoded], axis=1)

In [51]:
validation['ProductCD'].fillna(0,inplace=True)

In [52]:
P_emaildomain_encoded = (ohe.fit_transform(validation[['P_emaildomain']].astype(str)).todense().astype(int))
P_emaildomain_encoded = pd.DataFrame(P_emaildomain_encoded).add_prefix('P_emaildomain_')
train = pd.concat([validation, P_emaildomain_encoded], axis=1)

In [53]:
validation['P_emaildomain'].fillna(0,inplace=True)

In [54]:
card4_encoded = (ohe.fit_transform(validation[['card4']].astype(str)).todense().astype(int))
card4_encoded = pd.DataFrame(card4_encoded).add_prefix('card4')
train = pd.concat([validation, card4_encoded], axis=1)

In [55]:
validation['card4'].fillna(0,inplace=True)

In [56]:
for column in validation.columns:
    if validation[column].dtype==object:
        validation[column] = le.fit_transform(validation[column].astype(str))

In [57]:
validation.fillna(-1,inplace=True)

In [58]:
label2=validation['isFraud']

In [59]:
del(validation['isFraud'])

In [60]:
model.fit(validation, label2)

LogisticRegression(C=0.5)

In [61]:
print(classification_report(label2, model.predict(validation)))

              precision    recall  f1-score   support

           0       0.97      1.00      0.98    114044
           1       0.00      0.00      0.00      4064

    accuracy                           0.97    118108
   macro avg       0.48      0.50      0.49    118108
weighted avg       0.93      0.97      0.95    118108



C:\Users\julip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\julip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\julip\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [62]:
roc_auc_score(label2, model.predict_proba(validation)[:,1])

0.6686021252986484

### TEST

In [63]:
idenTest = pd.read_csv('test_identity.csv')

In [64]:
transTest = pd.read_csv('test_transaction.csv')

In [65]:
join = transTest.merge(idenTest,how='left')

In [66]:
join.head(2)

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [67]:
join.columns

Index(['TransactionID', 'TransactionDT', 'TransactionAmt', 'ProductCD',
       'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       ...
       'id-31', 'id-32', 'id-33', 'id-34', 'id-35', 'id-36', 'id-37', 'id-38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=433)

In [68]:
join

,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,...,id-31,id-32,id-33,id-34,id-35,id-36,id-37,id-38,DeviceType,DeviceInfo
0,3663549,18403224,31.950,W,10409,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.000,W,4272,111.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3663551,18403310,171.000,W,4476,574.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3663552,18403310,284.950,W,10989,360.0,150.0,visa,166.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3663553,18403317,67.950,W,18018,452.0,150.0,mastercard,117.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506686,4170235,34214279,94.679,C,13832,375.0,185.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506687,4170236,34214287,12.173,C,3154,408.0,185.0,mastercard,224.0,debit,...,chrome 43.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,ALE-L23 Build/HuaweiALE-L23
506688,4170237,34214326,49.000,W,16661,490.0,150.0,visa,226.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
506689,4170238,34214337,202.000,W,16621,516.0,150.0,mastercard,224.0,debit,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
id_34_encoded = (ohe.fit_transform(join[['id-34']].astype(str)).todense().astype(int))
id_34_encoded = pd.DataFrame(id_34_encoded).add_prefix('id_34_')
join = pd.concat([join, id_34_encoded], axis=1)

In [70]:
join['id-34'].fillna(0,inplace=True)

In [ ]:
join

In [71]:
id_35_encoded = (ohe.fit_transform(join[['id-35']].astype(str)).todense().astype(int))
id_35_encoded = pd.DataFrame(id_35_encoded).add_prefix('id_35_')
join = pd.concat([join, id_35_encoded], axis=1)

In [72]:
join['id-35'].fillna(0,inplace=True)

In [73]:
id_38_encoded = (ohe.fit_transform(join[['id-38']].astype(str)).todense().astype(int))
id_38_encoded = pd.DataFrame(id_38_encoded).add_prefix('id_38_')
join = pd.concat([join, id_38_encoded], axis=1)

In [74]:
join['id-38'].fillna(0,inplace=True)

In [75]:
ProductCD_encoded = (ohe.fit_transform(join[['ProductCD']].astype(str)).todense().astype(int))
ProductCD_encoded = pd.DataFrame(ProductCD_encoded).add_prefix('ProductCD_')
join = pd.concat([join, ProductCD_encoded], axis=1)

In [76]:
join['ProductCD'].fillna(0,inplace=True)

In [77]:
P_emaildomain_encoded = (ohe.fit_transform(join[['P_emaildomain']].astype(str)).todense().astype(int))
P_emaildomain_encoded = pd.DataFrame(P_emaildomain_encoded).add_prefix('P_emaildomain_')
join = pd.concat([join, P_emaildomain_encoded], axis=1)

In [78]:
join['P_emaildomain'].fillna(0,inplace=True)

In [79]:
card4_encoded = (ohe.fit_transform(join[['card4']].astype(str)).todense().astype(int))
card4_encoded = pd.DataFrame(card4_encoded).add_prefix('card4')
join = pd.concat([join, card4_encoded], axis=1)

In [80]:
join['card4'].fillna(0,inplace=True)

In [81]:
for column in join.columns:
    if join[column].dtype==object:
        join[column] = le.fit_transform(join[column].astype(str))

In [82]:
join.fillna(-1,inplace=True)

In [83]:
sub = pd.read_csv('sample_submission.csv')

In [84]:
sub.head(2)

,TransactionID,isFraud
0,3663549,0.5
1,3663550,0.5


In [85]:
join['predict'] = model.predict_proba(join)[:,1]

ValueError: X has 513 features per sample; expecting 433